## LangChain Chains

Chaining LLM tasks helps building advanced AI applications that can handle a sequence of tasks or resoning.  

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
from langchain import PromptTemplate, OpenAI, LLMChain

load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
template = """
Extract name of a person and language of message from the input.

Format the output as JSON with the following keys:
name
language

text: {input}
"""

llm = OpenAI(temperature=0)
prompt_template = PromptTemplate.from_template(template=template)
name_lang_chain = LLMChain(llm=llm, prompt=prompt_template)
name_lang_chain.predict(input="Herr Josef Braun ist am 22.09.1999 geboren.")

/tmp/ipykernel_19021/1338294985.py:11: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)
/tmp/ipykernel_19021/1338294985.py:13: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  name_lang_chain = LLMChain(llm=llm, prompt=prompt_template)


'\n{\n    "name": "Josef Braun",\n    "language": "German"\n}'

### Sequential Chain

In [5]:
response_template = """
You are an AI assistant generating greeting message for the beginning of an e-mail. 
Propose greeting using provided name and language.

text: {input}
"""
greeting_template = PromptTemplate(input_variables=["input"], template=response_template)
greeting_chain = LLMChain(llm=llm, prompt=greeting_template)
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[name_lang_chain, greeting_chain], verbose=True)

overall_chain.run(input="Herr Josef Braun ist am 22.09.1999 geboren.")

/tmp/ipykernel_19021/2838309493.py:13: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain.run(input="Herr Josef Braun ist am 22.09.1999 geboren.")




> Entering new SimpleSequentialChain chain...

{
    "name": "Josef Braun",
    "language": "German"
}

Guten Tag Josef Braun,

Ich hoffe, es geht Ihnen gut. Ich wollte Ihnen nur eine kurze E-Mail schreiben, um mich vorzustellen und Ihnen mitzuteilen, dass ich Ihr neuer AI-Assistent bin. Ich freue mich darauf, Ihnen bei all Ihren Aufgaben und Anfragen behilflich zu sein. Zögern Sie nicht, mich jederzeit zu kontaktieren.

Mit freundlichen Grüßen,
[Your Name]

> Finished chain.


'\nGuten Tag Josef Braun,\n\nIch hoffe, es geht Ihnen gut. Ich wollte Ihnen nur eine kurze E-Mail schreiben, um mich vorzustellen und Ihnen mitzuteilen, dass ich Ihr neuer AI-Assistent bin. Ich freue mich darauf, Ihnen bei all Ihren Aufgaben und Anfragen behilflich zu sein. Zögern Sie nicht, mich jederzeit zu kontaktieren.\n\nMit freundlichen Grüßen,\n[Your Name]'

## Custom callback

In [6]:
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


class CustomHandler(BaseCallbackHandler):
    buffer = []
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        self.buffer.append(token)
        if len(self.buffer) == 4:
            self.buffer.reverse() # inverse order of words to make text like said by master Yoda
            print(' '.join(self.buffer))
            self.buffer = []


prompt = ChatPromptTemplate.from_messages(["Answer a question{question}"])
model = ChatOpenAI(streaming=True, callbacks=[CustomHandler()])
chain = prompt | model

response = chain.invoke({"question": "What is the path of Jedi?"})

 of  path The 
 to  is  Jedi  a
 of  teachings  the  follow
,  Order  Jedi  the
 knowledge  seeking  includes  which
,  peace  maintaining ,
 and ,  justice  defending
 for  Force  the  using
.  good  greater  the
 to  strive  also  Jedi
 themselves  within  balance  maintain
,  universe  the  and
ations  tempt  the  resist
 side  dark  the  of
 the  uphold  and ,
,  compassion  of  principles
 and , lessness  self
. ipline -disc  self
 of  goal  ultimate  The
 to  is  Jedi  a
 the  with  one  become
 a  achieve  and  Force
 and  harmony  of  state
